modified from Song's `hsc_um2` repo

In [4]:
%load_ext line_profiler
%reload_ext autoreload
%autoreload 2

In [1]:
from functions import *

3.0rc1


In [12]:
config_initial = parse_config('/Users/fardila/Documents/GitHub/dwarf_lensing/MCMC/mcmc_test_config.yaml')
config, cosmos_data, sim_data = initial_model(config_initial)

3.0rc1
5SHMR+scatter


In [7]:
%lprun -f load_sim_data initial_model(config_initial)

In [27]:
emcee_fit(config, cosmos_data, sim_data)

3.0rc1
1 thread(s)
# Phase: Burn-in ...
[ 0.36674566  0.20175413 12.67483993 11.30854762  0.59999131  0.6117067
  1.4976905 ]
lnlike
mock.populate
SMF computed
cut galaxies table 52207
DS computed
model_prediced
smf_lnlike
dsigma_lnlike
[ 0.3709471   0.10288746 12.68879281 11.09944657  0.54624369  0.45596324
  1.46517504]
lnlike
mock.populate


  0%|          | 0/2 [00:00<?, ?it/s]

SMF computed
cut galaxies table 52626
DS computed
model_prediced
smf_lnlike
dsigma_lnlike
[ 0.3630513   0.28868867 12.66257101 11.49241244  0.64725217  0.74865361
  1.52628169]
lnlike
mock.populate
SMF computed
cut galaxies table 49886
DS computed
model_prediced
smf_lnlike
dsigma_lnlike
[ 0.37198919  0.0783652  12.69225359 11.04758248  0.53291247  0.41733363
  1.45711012]
lnlike
mock.populate


 50%|█████     | 1/2 [00:06<00:06,  6.32s/it]

SMF computed
cut galaxies table 51984
DS computed
model_prediced
smf_lnlike
dsigma_lnlike
[ 0.3642641   0.26014945 12.6665987  11.43205256  0.63173719  0.70369614
  1.51689566]
lnlike
mock.populate
SMF computed
cut galaxies table 50821
DS computed
model_prediced
smf_lnlike
dsigma_lnlike
[ 0.37294754  0.05581371 12.69543625 10.99988653  0.52065264  0.38180854
  1.44969334]
lnlike
mock.populate
SMF computed


...

 50%|█████     | 1/2 [00:08<00:08,  8.52s/it]

SMF computed
cut galaxies table 51787
DS computed
model_prediced
smf_lnlike
dsigma_lnlike
[ 0.37078202  0.10677197 12.68824459 11.10766224  0.54835546  0.46208247
  1.46645259]
lnlike
mock.populate
SMF computed
cut galaxies table 52949
DS computed
model_prediced
smf_lnlike
dsigma_lnlike
[ 0.37179573  0.08291767 12.69161111 11.05721087  0.53538737  0.42450508
  1.45860734]
lnlike
mock.populate
SMF computed


100%|██████████| 2/2 [00:15<00:00,  8.17s/it]

cut galaxies table 52406
DS computed
model_prediced
smf_lnlike
dsigma_lnlike
#------------------------------------------------------
#  Mean acceptance fraction 0.0
#------------------------------------------------------
#  Best ln(Probability):        -inf
[ 0.37060149  0.1110201  12.68764506 11.11664696  0.5506649   0.4687745
  1.46784972]
#------------------------------------------------------


State([[ 0.37060149  0.1110201  12.68764506 11.11664696  0.5506649   0.4687745
   1.46784972]
 [ 0.37231956  0.0705912  12.69335072 11.03114062  0.52868624  0.40508734
   1.45455339]], log_prob=[-inf -inf], blobs=None, random_state=('MT19937', array([ 470002714,  328914452, 1864429847, 2348213592, 2976716377,
       2924691557,  312837123, 3457053165, 4195392930, 1124829709,
       3148650652, 3014082594, 4243113384, 2960533228, 1467016987,
       2951279174, 2740585392, 1500096750,  212688574, 2571236445,
       1694888489, 2027011502,   80219271, 2018534473, 3591198359,
       3343088714, 3737999818,  293026708, 3896753995, 2732713773,
        970423075, 2773390522, 4010655800, 3770756873, 3766659081,
       3304254905, 2048009386,  246917274, 1027531068,  175152294,
       4131766616, 3616545338, 3758502777, 4072643079, 3048342820,
        508532756,  816971990,  695871773, 1937364729, 2769614417,
        674999038, 2661199809,  794859101, 2099371839, 1367117770,
       2790765426, 

In [5]:
mock_galaxies = sim_data['model'].mock.galaxy_table
mock_galaxies = mock_galaxies['x', 'y', 'z', 'stellar_mass']
mock_galaxies = np.array(mock_galaxies[(np.log10(mock_galaxies['stellar_mass'])>=min(cosmos_data['cosmos_dwarf_masses'])) & \
                              (np.log10(mock_galaxies['stellar_mass'])<9.0)])
# half_mock_galaxies = np.random.choice(mock_galaxies,500000)

In [11]:
len(galaxies_table) /50.

6876.0

test weird models

In [24]:
params = [ 0.14187446,  0.09982286, 12.94991423, 10.57434762,  0.41318031,  0.52853469,  1.4536998 ]
ln_prob_global(params, config, cosmos_data , sim_data)

3.0rc1
[0.14187446, 0.09982286, 12.94991423, 10.57434762, 0.41318031, 0.52853469, 1.4536998]
lnlike
mock.populate
SMF computed
cut galaxies table 1852
DS computed
model_prediced
smf_lnlike
dsigma_lnlike


-inf

In [ ]:
import time

time1 = time.time()

run_time = time.time() - time1
print('Total time: {0} seconds; {1} minutes; {2} hours'.format(run_time, run_time/60., run_time/3600. ))
